In [4]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import datasets
import pandas as pd
from sklearn.cluster import DBSCAN, KMeans
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# !pip install py7zr
# import py7zr

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install hdbscan
import hdbscan
def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.metrics import confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039221 sha256=70c164af2ea66db973285a16ecbcaa64ac62eff356464cdb4def520bcdebd4dc
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan


In [7]:
def calculate_f_measure(y,labels):
  cm = confusion_matrix(y, labels)
  indexes = linear_assignment(_make_cost_m(cm))
  # js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
  cm2 = cm[:, indexes[1]]

  acc = np.trace(cm2) / np.sum(cm2)
  precision = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 0))
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))
  f1 = 2 * precision * recall / (precision + recall)
  return f1

def evaluate_clustering(labels, predictions, beta=1.0):
  rand_index = metrics.rand_score(labels, predictions)
  nmi = metrics.normalized_mutual_info_score(labels, predictions)
  f_measure = calculate_f_measure(labels, predictions)
  print('Rand Index: %f, NMI: %f, F-measure: %f' % (rand_index, nmi, f_measure))
  return


In [8]:
#archive = py7zr.SevenZipFile('/content/face_encodings.7z', mode='r')
#archive.extractall(path="/content")
#archive.close()

# df = pd.read_csv('/content/face_encodings.csv')
df = pd.read_csv('/content/drive/MyDrive/face_encodings.csv')


y = df['target']

df = df.drop('target', axis=1)

In [6]:
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)
pca = PCA(n_components=100)
df = pca.fit_transform(df)

kmeans = KMeans(n_clusters=105,n_init=20, max_iter=500)

kmeans.fit(df)

labels = kmeans.labels_

evaluate_clustering(y,labels)

Rand Index: 0.995588, NMI: 0.894307, F-measure: 0.888649


In [10]:
%%time
min_cluster_size = 5  # Minimum number of samples in a cluster
min_samples = 3  # The minimum number of samples in a neighborhood for a point to be considered as a core point

hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
hdbscan_cluster_labels = hdbscan_model.fit_predict(df)

# Step 3: Find the number of clusters from HDBSCAN results
num_clusters_hdb = len(np.unique(hdbscan_cluster_labels)) - 1  # -1 to exclude the outliers (-1 label)
num_clusters_hdb

CPU times: user 1min 2s, sys: 192 ms, total: 1min 3s
Wall time: 1min 6s


112

In [8]:
evaluate_clustering( y,hdbscan_cluster_labels)

Rand Index: 0.959366, NMI: 0.811336, F-measure: 0.837322


<ipython-input-4-a9a7f9a9b946>:9: RuntimeWarning: invalid value encountered in true_divide
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))


In [9]:
kmeans = KMeans(n_clusters=num_clusters_hdb,n_init=20, max_iter=500)

kmeans.fit(df)

labels = kmeans.labels_

evaluate_clustering(y,labels)

Rand Index: 0.996146, NMI: 0.896861, F-measure: 0.900091


<ipython-input-4-a9a7f9a9b946>:9: RuntimeWarning: invalid value encountered in true_divide
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))


# GPU-Accelerated Hierarchical DBSCAN

In [1]:
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 957.1/957.1 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/

In [2]:
from gensim import models
# import cupy as cp

from cuml.manifold import UMAP
from cuml.preprocessing import normalize
from cuml.cluster import HDBSCAN

In [3]:
hdbscan_min_samples=3
hdbscan_min_cluster_size=5
hdbscan_max_cluster_size=1000

In [9]:
%%time
hdbscan = HDBSCAN(min_samples=hdbscan_min_samples,
                  min_cluster_size=hdbscan_min_cluster_size,
                  max_cluster_size=hdbscan_max_cluster_size)

labels = hdbscan.fit_predict(df)

CPU times: user 3.14 s, sys: 992 ms, total: 4.13 s
Wall time: 4.86 s


In [10]:
len(set(labels))

119

In [11]:
evaluate_clustering(y,labels)

Rand Index: 0.968826, NMI: 0.830277, F-measure: 0.828798


<ipython-input-7-a9a7f9a9b946>:9: RuntimeWarning: invalid value encountered in true_divide
  recall = np.nanmean(np.diag(cm2) / np.sum(cm2, axis = 1))
